# Import Libraries

In [61]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [2]:
# Reading the data
data = pd.read_excel('../data/responses_data.xlsx')

In [3]:
# Checking the head of the data
data.head()

,description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
2,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
3,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Racist
4,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent


In [6]:
# The shape of the data
data.shape

(2953, 2)

In [5]:
data.description = data.description.apply(str.strip)

In [7]:
# The columns of the data
data.columns

Index(['description', 'answer_category_num'], dtype='object')

In [47]:
data.description.duplicated().sum()

711

In [48]:
clean_data = data.copy()
clean_data  = clean_data[['description', 'answer_category_num']]
clean_data.head(2)

,description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation


In [92]:
clean_data.description.value_counts()

وزير الخارجية اللبناني جبران باسيل قال في سلسلة تغريداته عقب اختتام القمة العربية التنموية الاقتصادية والاجتماعية لمسنا تج…    7
سورية بلد الحضارات تربطها بعلية او بحيوان                                                                                      6
لما العاهرة تاضر بالعفة                                                                                                        6
بس بدك تحكي تحكيش متل الخنازير                                                                                                 5
أقترح ردم اللاجئين بالبحر على طريقة ردم جبل النفايات بصيدا                                                                     5
                                                                                                                              ..
انت كلب واطي بل أخلاق مجرم توأم بشار ألأسد كلب منحط غير درزي شركسي أرحل قبل أن نرحلك الي جهنم يا وليد حزا                      1
لو بتخرس ولا احلالك انقلع شقفة مجنس لاجئ عندنا                                                   

In [94]:
sum(clean_data.description.value_counts() > 1)

546

In [13]:
# removing duplicated descrptions/ comments
clean_data.drop_duplicates(subset='description', inplace=True)
clean_data.reset_index(drop=True, inplace=True)

In [14]:
clean_data

,description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent
2,تقتلون وسام الحسن وتترحموعلية من أي أصناف المخ...,Racist
3,معك خبر انو بلدة قطر متل ما سميتا مساحتها اكبر...,Normal
4,للامانه قوت الموسم اللي طاف كان هوا بس متحمس ح...,Normal
...,...,...
2237,هههه اخ وئام كون واقعي شوي انت بينك وبين حالك ...,Violent
2238,لقاء خلدة قادر ان يسمي شيخ فتنة او شيخ تحريض ا...,Religious affiliation
2239,اكثر ثلاث كلمات متداولة من الاكثر للاقل باسيل ...,Mockery
2240,اجواء اوروبيه بواقع صومالي زنجي حقير,Racist


In [15]:
# checking how many comments have different votes
for i, row in clean_data.iterrows():

    comment = row.description

    # getting the dataframe for that comment
    temp_df = data[data.description == comment]
    
    # how many unique answers that comment has
    n_ = temp_df.answer_category_num.nunique()

    # changing the answer_category_num to the most frequent one
    if n_ >= 2:
        most_voted_label = temp_df.answer_category_num.value_counts().index[0]
        clean_data.at[i, 'answer_category_num'] = most_voted_label

In [16]:
clean_data

,description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent
2,تقتلون وسام الحسن وتترحموعلية من أي أصناف المخ...,Racist
3,معك خبر انو بلدة قطر متل ما سميتا مساحتها اكبر...,Normal
4,للامانه قوت الموسم اللي طاف كان هوا بس متحمس ح...,Normal
...,...,...
2237,هههه اخ وئام كون واقعي شوي انت بينك وبين حالك ...,Violent
2238,لقاء خلدة قادر ان يسمي شيخ فتنة او شيخ تحريض ا...,Religious affiliation
2239,اكثر ثلاث كلمات متداولة من الاكثر للاقل باسيل ...,Mockery
2240,اجواء اوروبيه بواقع صومالي زنجي حقير,Racist


In [17]:
# Generated Class
clean_data.answer_category_num.value_counts()

Normal                   556
Mockery                  518
Violent                  410
Sexual harrasment        285
Religious affiliation    256
Racist                   217
Name: answer_category_num, dtype: int64

In [18]:
# Checking the info
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2242 entries, 0 to 2241
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   description          2242 non-null   object
 1   answer_category_num  2242 non-null   object
dtypes: object(2)
memory usage: 35.2+ KB


In [70]:
# Removing stop words for each tweet
clean_data['tweet_no_stopwords'] = 'x'
stop_words = set(stopwords.words('arabic')) 
for count, tweet in enumerate(clean_data.description):
    word_tokens = word_tokenize(tweet)
    filtered_tweet = []
    for word in word_tokens:
        if word not in stop_words:
            filtered_tweet.append(word)
    joined_filtered_tweet = " ".join(filtered_tweet)
    clean_data.tweet_no_stopwords[count] = joined_filtered_tweet

In [72]:
clean_data.head(2)

,description,answer_category_num,tweet_no_stopwords
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation,وزير الخارجية اللبناني جبران باسيل قال سلسلة ت...
1,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation,وزير الخارجية اللبناني جبران باسيل قال سلسلة ت...


# Training the Model

In [73]:
# y is Class which is dependent on X Tweet
X = clean_data['tweet_no_stopwords']
y = clean_data['answer_category_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [86]:
y_train.shape, y_test.shape

((2362,), (591,))

In [74]:
y_train.value_counts()

Normal                   552
Mockery                  548
Violent                  425
Religious affiliation    297
Sexual harrasment        291
Racist                   249
Name: answer_category_num, dtype: int64

In [75]:
y_test.value_counts()

Normal                   149
Mockery                  126
Violent                  104
Religious affiliation     84
Sexual harrasment         68
Racist                    60
Name: answer_category_num, dtype: int64

In [76]:
def show_results(y_true, y_pred):
    print("Confusion Matrix")
    print(metrics.confusion_matrix(y_true, y_pred))

    print("Classification Report")
    print(metrics.classification_report(y_true, y_pred)) 

In [77]:
# Using pipiles for machine learning flow

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [78]:
show_results(y_test, predictions)

Confusion Matrix
[[39 35  9 12  8 23]
 [26 68 11 15 13 16]
 [ 9 15  7  9  5 15]
 [19 25  5  9  8 18]
 [17 12  7 11  7 14]
 [27 24  6 11 10 26]]
Classification Report
                       precision    recall  f1-score   support

              Mockery       0.28      0.31      0.30       126
               Normal       0.38      0.46      0.41       149
               Racist       0.16      0.12      0.13        60
Religious affiliation       0.13      0.11      0.12        84
    Sexual harrasment       0.14      0.10      0.12        68
              Violent       0.23      0.25      0.24       104

             accuracy                           0.26       591
            macro avg       0.22      0.22      0.22       591
         weighted avg       0.25      0.26      0.25       591



In [79]:
# Using pipiles for machine learning flow
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [80]:
show_results(y_test, predictions)

Confusion Matrix
[[68 38  1  1  2 16]
 [39 93  0  4  4  9]
 [13 26  4  3  1 13]
 [32 40  0  1  1 10]
 [31 21  0  3  2 11]
 [40 36  0  2  2 24]]
Classification Report
                       precision    recall  f1-score   support

              Mockery       0.30      0.54      0.39       126
               Normal       0.37      0.62      0.46       149
               Racist       0.80      0.07      0.12        60
Religious affiliation       0.07      0.01      0.02        84
    Sexual harrasment       0.17      0.03      0.05        68
              Violent       0.29      0.23      0.26       104

             accuracy                           0.32       591
            macro avg       0.33      0.25      0.22       591
         weighted avg       0.32      0.32      0.27       591



In [89]:
tst =  "انت  واحد حقير وما بتستحي ا"

In [90]:
text_clf.predict([tst])[0]

'Violent'

In [88]:
wrong = 0
mock_viol = 0

mock_viol_list = ['Violent', 'Mockery']

for count, test_tweet in enumerate(X_test):
    
    y_test_answer = y_test.values[count]
    y_test_prediction = predictions[count]

    if y_test_answer != y_test_prediction:
        print("Tweet: {}".format(test_tweet))
        print("Real Prediction: {} ||  Model Prediction: {}\n".format(y_test_answer, y_test_prediction))

        if y_test_answer in mock_viol_list and y_test_prediction in mock_viol_list:
            mock_viol += 1
        wrong += 1
        
print("Misclassified {}".format(wrong))
mock_viol

Tweet: الدولة اللبنانية وأقطابها تريد إزالة إسرائيل الوجود تريد السلام العادل والشامل مبادرة بيروت
Real Prediction: Normal ||  Model Prediction: Religious affiliation

Tweet: دخلك شيخ عقل بلبنان بهمتكم؟
Real Prediction: Mockery ||  Model Prediction: Violent

Tweet: شاغل الفاسدين والعاهرين ملك جبران باسيل العدرا تحميك ❤
Real Prediction: Religious affiliation ||  Model Prediction: Normal

Tweet: الوزير جبران باسيل الافضل لمحاربة الفساد
Real Prediction: Racist ||  Model Prediction: Normal

Tweet: استاذ وئام انت رجل كبير ولك تقديرك نحب ان نجرحك بكلمة سيدي سؤال ايش حشرك دول أخرى
Real Prediction: Sexual harrasment ||  Model Prediction: Mockery

Tweet: مش دايماً الولاد بيشبهوا أهلُن لأن أوقات بشوف أمهات شفافن دافشين وولادن شفافن مسح 😎
Real Prediction: Violent ||  Model Prediction: Normal

Tweet: تعلم ان جبران باسيل يعلم
Real Prediction: Mockery ||  Model Prediction: Normal

Tweet: حضرتك شو بتشتغل قصاب بأي قنصلية كوليرا
Real Prediction: Sexual harrasment ||  Model Prediction: Mockery

Tweet: ا

56

# Saving the Model

In [27]:
import pickle
# save the model to disk
filename = 'finalized_model_SVC.sav'
pickle.dump(text_clf, open(filename, 'wb'))

In [74]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8393162393162393
